In [1]:
import pandas as pd
import nltk
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

In [2]:
# Download Necessary NLTK Data Packages

# Downloading NLTK data packages
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HakobAvjyan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HakobAvjyan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HakobAvjyan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\HakobAvjyan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
# Load the CVE Data
cve_data_path = r'C:\Users\HakobAvjyan\new_cve_data.csv'
cve_data = pd.read_csv(cve_data_path)

In [4]:
# Define the preprocess_text function
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens


In [5]:
# Preprocess the CVE Descriptions
cve_data['processed_desc'] = cve_data['Description'].apply(preprocess_text)

In [6]:
# Step 3: Apply Topic Modeling for CVE Topic Name
dictionary = corpora.Dictionary(cve_data['processed_desc'])
corpus = [dictionary.doc2bow(text) for text in cve_data['processed_desc']]

# Train the LDA model
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, random_state=42)

# Assign Topic Names to the Topics from LDA
topic_names = {
    0: "Remote Code Execution",
    1: "Privilege Escalation",
    2: "Information Disclosure",
    3: "Denial of Service",
    4: "SQL Injection",
    5: "Cross-Site Scripting",
    6: "Buffer Overflow",
    7: "Unauthorized Access",
    8: "Malware",
    9: "Network Exploitation"
}

def get_topic_name(lda_model, corpus, topic_names):
    topics = lda_model[corpus]
    dominant_topic = max(topics, key=lambda item: item[1])[0]
    return topic_names[dominant_topic]

# Apply the topic model to each CVE description and assign the corresponding topic name
cve_data['cve_topic_name'] = [get_topic_name(lda_model, doc, topic_names) for doc in corpus]

In [7]:
# Step 4: Load and Parse the NIST Control Catalog with Enhancements
control_catalog_path = r'C:\Users\HakobAvjyan\sp800-53r5-control-catalog.xlsx'
control_catalog = pd.read_excel(control_catalog_path)

# Extract control identifiers and names, ensuring enhancements are included
control_catalog = control_catalog[['Control Identifier', 'Control (or Control Enhancement) Name', 'Control Text', 'Discussion', 'Related Controls']]

# Create a dictionary to map control identifiers to their full details (including enhancements)
control_dict = control_catalog.set_index('Control Identifier').T.to_dict('index')

In [8]:
# Step 5: Define the Keyword-to-NIST Control Mapping (Expanded Version with Enhancements)
keyword_to_nist = {
    # Access Control
    'authentication': ['AC-2', 'AC-2(1)', 'IA-2', 'IA-2(1)', 'IA-2(2)'],
    'access control': ['AC-3', 'AC-3(1)', 'AC-6', 'AC-6(1)'],
    'authorization': ['AC-3', 'AC-3(1)', 'AC-6'],
    'privilege escalation': ['AC-6', 'AC-6(1)', 'AC-14'],
    'unauthorized access': ['AC-3', 'AC-6'],
    'least privilege': ['AC-6', 'AC-6(1)'],
    'remote': ['AC-17', 'AC-17(1)', 'SC-7'],  # Related to remote access and control
    'local': ['AC-6', 'AC-6(1)', 'CM-6'],  # Local privilege escalation or local access control
    'privileges': ['AC-6', 'AC-14', 'AC-14(1)'],  # Privilege escalation or unauthorized privilege assignment
    'password': ['IA-5', 'IA-5(1)', 'IA-5(2)'],
    'weak password': ['IA-5', 'IA-5(1)', 'IA-2'],
    'default credentials': ['IA-5', 'IA-5(1)', 'IA-5(2)'],
    'biometric': ['IA-2', 'IA-2(1)', 'IA-2(2)'],
    'multi-factor authentication': ['IA-2', 'IA-2(1)', 'IA-2(2)'],
    'csrf': ['AC-2', 'AC-2(1)', 'IA-2', 'IA-2(1)', 'IA-2(2)'],  # Cross-site request forgery

    # System and Communications Protection
    'encryption': ['SC-12', 'SC-12(1)', 'SC-28', 'SC-28(1)'],
    'cryptographic': ['SC-12', 'SC-12(1)', 'SC-13', 'SC-13(1)', 'SC-28'],
    'tls': ['SC-8', 'SC-8(1)', 'SC-13'],
    'ssl': ['SC-8', 'SC-8(1)', 'SC-13'],
    'firewall': ['SC-7', 'SC-7(1)'],
    'vpn': ['SC-8', 'SC-8(1)'],
    'network segmentation': ['SC-7', 'SC-7(1)'],
    'malware': ['SI-3', 'SI-3(1)', 'SI-3(2)'],
    'virus': ['SI-3', 'SI-3(1)', 'SI-3(2)'],
    'worm': ['SI-3', 'SI-3(1)', 'SI-3(2)'],
    'trojan': ['SI-3', 'SI-3(1)', 'SI-3(2)'],
    'buffer overflow': ['SI-16'],
    'sql injection': ['SI-10', 'SI-10(1)', 'SI-10(2)'],
    'cross-site scripting': ['SI-10', 'SI-10(1)'],
    'input validation': ['SI-10', 'SI-10(1)'],
    'data corruption': ['SI-7', 'SI-16'],  # Expanded from original
    'tampering': ['SI-7', 'SI-7(1)'],
    'arbitrary': ['SI-10', 'SI-3', 'SI-3(1)', 'SI-3(2)'],  # Arbitrary code execution, often related to input validation or malicious code protection
    'crafted': ['SI-10', 'SI-10(1)'],  # Crafted inputs often relate to input validation issues
    'code': ['SI-7', 'SI-10'],  # Code execution issues often related to integrity
    'execute': ['SI-7'],  # Execution of code, especially arbitrary or remote
    'memory': ['SI-16'],  # Memory corruption, often related to buffer overflow protections
    'corruption': ['SI-7', 'SI-16'],  # Data corruption, related to system and information integrity
    'xss': ['SI-10'],  # Cross-site scripting (XSS)
    'sqli': ['SI-10'],  # SQL injection
    'application': ['SA-22'],  # Application development and security acquisition
    'service': ['SC-5'],  # Service disruptions, related to denial of service protection
    'file': ['MP-5'],  # File access, storage, and protection
    'server': ['SC-7'],  # Server issues often relate to boundary protection
    'web': ['SI-10', 'SC-7'],  # Web applications, often related to input validation and boundary protection
    'flash': ['SI-3'],  # Specific to Flash Player vulnerabilities, likely involving malicious code protection
    'crash': ['SI-7'],  # Crashes often relate to system integrity
    'read': ['SC-28'],  # Unauthorized read access, related to data protection
    'sensitive': ['SC-28'],  # Sensitive data, related to protection of information at rest
    'information': ['SC-28'],  # General information protection
    'adobe': ['SI-3'],  # Adobe-related vulnerabilities, likely involving malicious code protection
    'air': ['SI-3'],  # Adobe AIR-specific vulnerabilities, likely involving malicious code protection

    # Incident Response
    'incident response': ['IR-4'],
    'intrusion detection': ['IR-4', 'SI-4'],
    'security event': ['IR-4', 'AU-6'],
    'breach': ['IR-4'],

    # Audit and Accountability
    'logging': ['AU-2', 'AU-2(1)', 'AU-3'],
    'audit log': ['AU-2', 'AU-2(1)', 'AU-3'],
    'monitoring': ['AU-6', 'SI-4'],
    'audit trail': ['AU-3'],

    # Contingency Planning
    'backup': ['CP-9', 'CP-9(1)'],
    'disaster recovery': ['CP-10'],
    'data recovery': ['CP-9', 'CP-9(1)', 'CP-10'],
    'business continuity': ['CP-11'],

    # Configuration Management
    'misconfiguration': ['CM-6', 'CM-6(1)', 'CM-7'],
    'default settings': ['CM-6', 'CM-7'],
    'configuration baseline': ['CM-2', 'CM-2(1)'],
    'properly': ['CM-6'],  # Misconfiguration or improper handling
    'linux': ['CM-6'],  # Linux-specific configuration issues
    'windows': ['CM-6', 'SC-7'],  # Windows-specific configuration issues
    'kernel': ['CM-6'],  # Kernel-level issues often relate to configuration management

    # Maintenance
    'system updates': ['MA-2', 'MA-2(1)'],
    'patch management': ['MA-2', 'MA-2(1)', 'SI-2'],
    'software update': ['MA-2', 'MA-2(1)', 'SI-2'],

    # Physical and Environmental Protection
    'physical access': ['PE-3'],
    'environmental controls': ['PE-14'],
    'cctv': ['PE-6'],

    # System and Services Acquisition
    'supply chain': ['SA-12', 'SA-12(1)'],
    'third-party': ['SA-9'],
    'vendor management': ['SA-9'],

    # Awareness and Training
    'security training': ['AT-2', 'AT-2(1)'],
    'awareness training': ['AT-2'],
    'role-based training': ['AT-3'],

    # Personnel Security
    'insider threat': ['PS-7'],
    'background check': ['PS-3'],

    # Risk Assessment
    'vulnerability assessment': ['RA-5'],
    'risk assessment': ['RA-3'],
    'security assessment': ['CA-2'],

    # Media Protection
    'data sanitization': ['MP-6'],
    'media disposal': ['MP-6'],
    'data encryption': ['MP-5'],

    # Additional terms from topic modeling
    'attacker': ['SC-7'],  # Attacker actions are typically related to boundary and access controls
    'via': ['SC-7'],  # Actions performed via some medium often relate to boundary protection
    'obtain': ['IA-5'],  # Obtaining sensitive data often relates to authentication controls
    'function': ['SI-7'],  # Related to software integrity issues
    'exploit': ['SI-10'],  # Often associated with input validation or code integrity issues
    'vulnerability': ['SI-2'],  # General vulnerability management and mitigation
    'telemetry': ['SI-4'],  # Telemetry data often relates to monitoring and detection
    'setuid': ['AC-6'],  # Setuid issues often relate to privilege escalation
    'station': ['SC-7'],  # Stations often involve boundary protection
    'sdk': ['SA-22'],  # SDK issues often relate to application development and security acquisition
    'wild': ['SI-3'],  # Related to malicious code protection, particularly in wild exploits
    'flash': ['SI-3'],  # Flash Player vulnerabilities involving malicious code
}


In [9]:
# Step 6: Further Enhanced Mapping Function for CVSS and Description-Based Mapping
def further_enhanced_map_to_nist_controls(row):
    controls = set()  # Using a set to avoid duplicate controls

    # Mapping based on Access Vector
    if row['Access Vector'] == 'NETWORK':
        controls.add('SC-7')
        controls.add('AC-17')
        controls.add('SC-7(3)')

    # Mapping based on Confidentiality Impact
    if row['Confidentiality Impact'] in ['PARTIAL', 'COMPLETE']:
        controls.add('SC-28')
        controls.add('SC-28(1)')
        controls.add('AC-3')
        controls.add('AC-3(3)')
        controls.add('IA-8')
        controls.add('IA-8(1)')

    # Mapping based on Integrity Impact
    if row['Integrity Impact'] in ['PARTIAL', 'COMPLETE']:
        controls.add('SI-7')
        controls.add('SI-7(1)')
        controls.add('SI-2')
        controls.add('SI-2(2)')
        controls.add('SA-11')
        controls.add('SA-11(1)')

    # Mapping based on Availability Impact
    if row['Availability Impact'] in ['PARTIAL', 'COMPLETE']:
        controls.add('CP-10')
        controls.add('CP-10(1)')
        controls.add('IR-4')
        controls.add('IR-4(1)')
        controls.add('SI-4')
        controls.add('SI-4(1)')

    # Additional mappings based on Authentication requirements
    if row['Authentication'] == 'NONE':
        controls.add('IA-2')
        controls.add('IA-2(1)')
        controls.add('IA-8')
        controls.add('IA-8(1)')

    # Example mapping based on the CVSS Base Score
    if row['CVSS Base Score'] >= 7.0:
        controls.add('RA-5')
        controls.add('RA-5(1)')
        controls.add('PM-9')
        controls.add('PM-9(1)')

    # Mapping for vulnerabilities that might involve remote access or configuration changes
    if 'remote' in row['Description'].lower():
        controls.add('AC-17')
        controls.add('AC-17(1)')
    if 'configuration' in row['Description'].lower():
        controls.add('CM-3')
        controls.add('CM-3(1)')

    # Mapping for vulnerabilities related to malicious code or monitoring
    if 'malicious' in row['Description'].lower() or 'code' in row['Description'].lower():
        controls.add('SI-3')
        controls.add('SI-3(1)')
        controls.add('SI-4')
        controls.add('SI-4(1)')

    # Mapping for vulnerabilities that could involve physical access
    if 'physical' in row['Description'].lower():
        controls.add('PE-3')
        controls.add('PE-3(1)')

    return list(controls)  # Convert set to list for easier handling

In [11]:
# Step 7: Combined Mapping Function
def combined_map_to_nist_controls(row):
    controls = set()

    # Keyword-based mapping
    for keyword, nist_controls in keyword_to_nist.items():
        if keyword in row['Description'].lower():
            controls.update(nist_controls)

    # Further enhanced mapping based on CVSS metrics and description
    enhanced_controls = further_enhanced_map_to_nist_controls(row)
    controls.update(enhanced_controls)

    return list(controls)

# Apply the Combined Mapping Function to the CVE Data
cve_data['Mapped_NIST_Controls'] = cve_data.apply(combined_map_to_nist_controls, axis=1)

In [12]:
# Step 8: Organize the Data into the Desired Format
def get_control_details(control_ids):
    descriptions = []
    discussions = []
    related_texts = []
    for control_id in control_ids:
        if control_id in control_dict:
            control_info = control_dict[control_id]
            formatted_description = f"{control_id}:\n{control_info['Control Text'].replace(';', ';' + chr(10))}"  # Replacing with newline safely
            descriptions.append(formatted_description)
            discussions.append(f"{control_id}:\n{control_info['Discussion']}")
            related_controls = control_info['Related Controls']
            if pd.isna(related_controls) or related_controls.strip() == "":
                related_controls = "None."
            formatted_related_text = f"{control_id}:\n{related_controls.replace(';', ', ')}"
            related_texts.append(formatted_related_text)
        else:
            descriptions.append(f"{control_id}: Not found")
            discussions.append(f"{control_id}: Not found")
            related_texts.append(f"{control_id}: Not found")
    return "\n\n".join(descriptions), "\n\n".join(discussions), "\n\n".join(related_texts)


In [13]:
# Apply the function to get control descriptions, discussions, and related text
cve_data[['potential_control_match_descriptions', 
          'potential_control_match_discussions', 
          'potential_control_match_related_text']] = cve_data['Mapped_NIST_Controls'].apply(lambda controls: pd.Series(get_control_details(controls)))


In [14]:
print(cve_data.columns)

Index(['CVE ID', 'Description', 'CVSS Base Score', 'Access Vector',
       'Access Complexity', 'Authentication', 'Confidentiality Impact',
       'Integrity Impact', 'Availability Impact', 'References',
       'processed_desc', 'cve_topic_name', 'Mapped_NIST_Controls',
       'potential_control_match_descriptions',
       'potential_control_match_discussions',
       'potential_control_match_related_text'],
      dtype='object')


In [15]:
# Save the data
output_file_path = r'C:\Users\HakobAvjyan\cve_data_complete.csv'
cve_data.to_csv(output_file_path, index=False)

print(f"cve_data saved successfully to {output_file_path}")

cve_data saved successfully to C:\Users\HakobAvjyan\cve_data_complete.csv
